In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load combined probabilities from CSV
combined_csv_path = 'all_probabilities_1.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)  # 5
num_samples = len(prob_df) // 3  # Assuming each model has predictions for all samples

# --- Data Preparation ---
print("=== Data Preparation ===")
print(f"Total samples: {num_samples}")
print(f"Number of classes: {num_classes}")
print(f"CSV shape: {prob_df.shape}")

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

print(f"True labels aligned: First 5 samples = {true_classes[:5]}")

# --- Model Probabilities ---
print("\n=== Model Probabilities ===")
model_probs = {}
for model_name in ['DenseNet169', 'InceptionV3', 'Xception']:
    model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
    probs = model_df[class_labels].values
    model_probs[model_name] = probs
    print(f"{model_name}:")
    print(f"  Shape: {probs.shape}")

# --- Fuzzy Ensembling ---
print("\n=== Fuzzy Ensembling Results ===")
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)
def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))
def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# Step 1: Compute membership scores for each model
model_scores = {model: compute_final_score(model_probs[model]) for model in model_probs}

# Step 2: Sum membership scores across models
ensemble_class_scores = np.zeros((num_samples, num_classes))
for model_name in model_scores:
    ensemble_class_scores += model_scores[model_name]

# Step 3: Predict classes based on summed scores
ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
print(f"First 5 predicted classes: {ensemble_predicted_classes[:5]}")

# --- Performance Evaluation ---
print("\n=== Performance Evaluation ===")
print("Classification Report:")
print(classification_report(true_classes, ensemble_predicted_classes, target_names=class_labels))

# --- Save Results ---
ensemble_df = pd.DataFrame({
    'True_Label': true_classes,
    'Predicted_Label': ensemble_predicted_classes
})
ensemble_df.to_csv('ensemble_predictions.csv', index=False)
print("\n=== Files Saved ===")
print("Predictions saved to: /content/ensemble_predictions.csv")


=== Data Preparation ===
Total samples: 810
Number of classes: 5
CSV shape: (2430, 8)
True labels aligned: First 5 samples = [0 1 4 1 1]

=== Model Probabilities ===
DenseNet169:
  Shape: (810, 5)
InceptionV3:
  Shape: (810, 5)
Xception:
  Shape: (810, 5)

=== Fuzzy Ensembling Results ===
First 5 predicted classes: [0 0 4 1 1]

=== Performance Evaluation ===
Classification Report:
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.91      0.93      0.92       163
            im_Koilocytotic       0.87      0.85      0.86       165
             im_Metaplastic       0.89      0.89      0.89       159
               im_Parabasal       0.92      0.91      0.92       157
im_Superficial-Intermediate       0.98      0.98      0.98       166

                   accuracy                           0.91       810
                  macro avg       0.91      0.91      0.91       810
               weighted avg       0.91      0.91      0.91    

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load combined probabilities from CSV
combined_csv_path = 'all_probabilities_1.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)  # 5
num_samples = len(prob_df) // 3  # Assuming each model has predictions for all samples

# --- Data Preparation ---
print("=== Data Preparation ===")
print(f"Total samples: {num_samples}")
print(f"Number of classes: {num_classes}")
print(f"CSV shape: {prob_df.shape}")

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

print(f"True labels aligned: First 5 samples = {true_classes[:5]}")

# --- Model Probabilities ---
print("\n=== Model Probabilities ===")
model_probs = {}
for model_name in ['DenseNet169', 'InceptionV3', 'Xception']:
    model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
    probs = model_df[class_labels].values
    model_probs[model_name] = probs
    print(f"{model_name}:")
    print(f"  Shape: {probs.shape}")

# --- Fuzzy Ensembling ---
print("\n=== Fuzzy Ensembling Results ===")
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)
def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))
def compute_final_score(probs):
    return expo_equ(probs) * norm_equ(probs)

# Step 1: Compute membership scores for each model
model_scores = {model: compute_final_score(model_probs[model]) for model in model_probs}

# Step 2: Sum membership scores across models
ensemble_class_scores = np.zeros((num_samples, num_classes))
for model_name in model_scores:
    ensemble_class_scores += model_scores[model_name]

# Step 3: Predict classes based on summed scores
ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
print(f"First 5 predicted classes: {ensemble_predicted_classes[:5]}")

# --- Performance Evaluation ---
print("\n=== Performance Evaluation ===")
print("Classification Report:")
print(classification_report(true_classes, ensemble_predicted_classes, target_names=class_labels))

# --- Save Results ---
ensemble_df = pd.DataFrame({
    'True_Label': true_classes,
    'Predicted_Label': ensemble_predicted_classes
})
ensemble_df.to_csv('ensemble_predictions.csv', index=False)
print("\n=== Files Saved ===")
print("Predictions saved to: /content/ensemble_predictions.csv")


=== Data Preparation ===
Total samples: 810
Number of classes: 5
CSV shape: (2430, 8)
True labels aligned: First 5 samples = [0 1 4 1 1]

=== Model Probabilities ===
DenseNet169:
  Shape: (810, 5)
InceptionV3:
  Shape: (810, 5)
Xception:
  Shape: (810, 5)

=== Fuzzy Ensembling Results ===
First 5 predicted classes: [0 0 4 1 1]

=== Performance Evaluation ===
Classification Report:
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.91      0.93      0.92       163
            im_Koilocytotic       0.86      0.85      0.86       165
             im_Metaplastic       0.89      0.89      0.89       159
               im_Parabasal       0.92      0.91      0.91       157
im_Superficial-Intermediate       0.98      0.98      0.98       166

                   accuracy                           0.91       810
                  macro avg       0.91      0.91      0.91       810
               weighted avg       0.91      0.91      0.91    

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load combined probabilities from CSV
combined_csv_path = 'all_probabilities_1.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)  # 5
num_samples = len(prob_df) // 3  # Assuming each model has predictions for all samples

# --- Data Preparation ---
print("=== Data Preparation ===")
print(f"Total samples: {num_samples}")
print(f"Number of classes: {num_classes}")
print(f"CSV shape: {prob_df.shape}")

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

print(f"True labels aligned: First 5 samples = {true_classes[:5]}")

# --- Model Probabilities ---
print("\n=== Model Probabilities ===")
model_probs = {}
for model_name in ['DenseNet169', 'InceptionV3', 'Xception']:
    model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
    probs = model_df[class_labels].values
    model_probs[model_name] = probs
    print(f"{model_name}:")
    print(f"  Shape: {probs.shape}")

# --- Fuzzy Ensembling ---
print("\n=== Fuzzy Ensembling Results ===")
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)
def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))
def compute_final_score(probs):
    return tanh_equ(probs) * norm_equ(probs)

# Step 1: Compute membership scores for each model
model_scores = {model: compute_final_score(model_probs[model]) for model in model_probs}

# Step 2: Sum membership scores across models
ensemble_class_scores = np.zeros((num_samples, num_classes))
for model_name in model_scores:
    ensemble_class_scores += model_scores[model_name]

# Step 3: Predict classes based on summed scores
ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
print(f"First 5 predicted classes: {ensemble_predicted_classes[:5]}")

# --- Performance Evaluation ---
print("\n=== Performance Evaluation ===")
print("Classification Report:")
print(classification_report(true_classes, ensemble_predicted_classes, target_names=class_labels))

# --- Save Results ---
ensemble_df = pd.DataFrame({
    'True_Label': true_classes,
    'Predicted_Label': ensemble_predicted_classes
})
ensemble_df.to_csv('ensemble_predictions.csv', index=False)
print("\n=== Files Saved ===")
print("Predictions saved to: /content/ensemble_predictions.csv")


=== Data Preparation ===
Total samples: 810
Number of classes: 5
CSV shape: (2430, 8)
True labels aligned: First 5 samples = [0 1 4 1 1]

=== Model Probabilities ===
DenseNet169:
  Shape: (810, 5)
InceptionV3:
  Shape: (810, 5)
Xception:
  Shape: (810, 5)

=== Fuzzy Ensembling Results ===
First 5 predicted classes: [4 4 3 4 3]

=== Performance Evaluation ===
Classification Report:
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.00      0.00      0.00     163.0
            im_Koilocytotic       0.00      0.00      0.00     165.0
             im_Metaplastic       0.00      0.00      0.00     159.0
               im_Parabasal       0.00      0.00      0.00     157.0
im_Superficial-Intermediate       0.00      0.00      0.00     166.0

                   accuracy                           0.00     810.0
                  macro avg       0.00      0.00      0.00     810.0
               weighted avg       0.00      0.00      0.00    

=== Data Preparation ===
Total samples: 810
Number of classes: 5
CSV shape: (4050, 8)
True labels aligned: First 5 samples = [0 1 4 1 1]

=== Model Probabilities ===
DenseNet201:
  Shape: (810, 5)
InceptionResNetV2:
  Shape: (810, 5)
Enhanced_VGG19:
  Shape: (810, 5)
MobileNetV2:
  Shape: (810, 5)
ResNet50V2:
  Shape: (810, 5)

=== Fuzzy Ensembling Results ===
First 5 predicted classes: [0 1 4 1 1]

=== Performance Evaluation ===
Classification Report:
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.93      0.98      0.96       163
            im_Koilocytotic       0.91      0.88      0.89       165
             im_Metaplastic       0.94      0.92      0.93       159
               im_Parabasal       0.97      0.95      0.96       157
im_Superficial-Intermediate       0.97      0.98      0.98       166

                   accuracy                           0.94       810
                  macro avg       0.94      0.94      0.94

In [10]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load combined probabilities from CSV
combined_csv_path = 'all_model_probabilities.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)
num_samples = len(prob_df) // 8  # Assuming each model has predictions for all samples

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

# Available models
model_list = ['DenseNet169', 'InceptionV3', 'Xception', 'DenseNet201', 'InceptionResNetV2', 'Enhanced_VGG19', 'MobileNetV2', 'ResNet50V2']

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# Generate all model combinations (starting from 2 models)
for r in range(2, len(model_list) + 1):  # r = 2 to 8
    for model_combo in itertools.combinations(model_list, r):
        print(f"\n=== Evaluating Combination: {model_combo} ===")
        
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {model: compute_final_score(model_probs[model]) for model in model_probs}
        
        # Sum membership scores across selected models
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        print(f"First 5 predicted classes: {ensemble_predicted_classes[:5]}")
        
        # Performance evaluation
        print("\n=== Performance Evaluation ===")
        print(classification_report(true_classes, ensemble_predicted_classes, target_names=class_labels))
        
        # Save results
        result_filename = f"ensemble_predictions_{'_'.join(model_combo)}.csv"
        ensemble_df = pd.DataFrame({
            'True_Label': true_classes,
            'Predicted_Label': ensemble_predicted_classes
        })
        ensemble_df.to_csv(result_filename, index=False)
        print(f"Predictions saved to: {result_filename}")



=== Evaluating Combination: ('DenseNet169', 'InceptionV3') ===
First 5 predicted classes: [0 0 4 1 1]

=== Performance Evaluation ===
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.89      0.94      0.92       163
            im_Koilocytotic       0.84      0.83      0.84       165
             im_Metaplastic       0.90      0.88      0.89       159
               im_Parabasal       0.94      0.92      0.93       157
im_Superficial-Intermediate       0.98      0.98      0.98       166

                   accuracy                           0.91       810
                  macro avg       0.91      0.91      0.91       810
               weighted avg       0.91      0.91      0.91       810

Predictions saved to: ensemble_predictions_DenseNet169_InceptionV3.csv

=== Evaluating Combination: ('DenseNet169', 'Xception') ===
First 5 predicted classes: [0 0 4 1 1]

=== Performance Evaluation ===
                             precision